# Load Dependencies

In [ ]:
import os
os.environ['WANDB_DISABLED']="true"

In [ ]:
!pip install evaluate

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

### Load Dataset

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

### Load Model

In [ ]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Function to count trainable and total parameters
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0

    # Loop through all model parameters
    for _, param in model.named_parameters():
        all_model_params += param.numel()           # total params
        if param.requires_grad:                    # check if trainable
            trainable_model_params += param.numel()


    return f"trainable model parameters: {trainable_model_params}\n" \
           f"all model parameters: {all_model_params}\n" \
           f"percentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"


print(print_number_of_trainable_model_parameters(original_model))


## Test the Model with Zero Shot Inferencing

In [ ]:
# Pick a test sample by index
index = 200

# Extract dialogue and human-written summary
dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

# Create summarization prompt
prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

# Tokenize the prompt into model inputs
inputs = tokenizer(prompt, return_tensors='pt')

# Generate summary using the model
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,     # limit output length
    )[0],
    skip_special_tokens=True   # clean special tokens
)

# Formatting line for readability
dash_line = '-'.join('' for x in range(100))

# Print input prompt, human summary, and model output
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')


# Full Fine Tuning

In [ ]:
# Function to tokenize dialogue-summary pairs
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '

    # Build input prompt for each dialogue
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]

    # Tokenize inputs (prompts) and labels (summaries)
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# Apply tokenization to all dataset splits (train/val/test)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Remove unused columns, keeping only tokenized data
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary'])

In [ ]:
# Take first tokenized input (prompt + dialogue)
sample_input_id = tokenized_datasets['train']['input_ids'][0:1]

# Take first tokenized label (summary)
sample_label = tokenized_datasets['train']['labels'][0:1]

# Print dataset object details
print("tokenized_datasets: ", tokenized_datasets)

# Print sample input ID length and values
print("\nsample_input_id: ", len(sample_input_id[0]), sample_input_id)

# Print sample label length and values
print("\nsample_label: ", len(sample_label[0]), sample_label)

In [ ]:
# Print dataset shapes (rows, columns) for each split
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

# Print tokenized dataset object summary
print(tokenized_datasets)


## Fine-Tune the Model with the Preprocessed Dataset

In [ ]:
# Create unique output directory using current timestamp
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

# Define training hyperparameters and logging settings
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    logging_steps=50,
    eval_strategy="epoch",   # <-- correct name in your version
    save_strategy="epoch",
)

# Initialize Trainer with model, args, and datasets
trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [ ]:
trainer.train()

In [ ]:
# Save the fully fine-tuned model
trainer.model.save_pretrained(output_dir)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load true original (pretrained) for baseline comparison
original_model = AutoModelForSeq2SeqLM.from_pretrained(
    "google/flan-t5-base", torch_dtype="bfloat16"
).to(device).eval()

# Load your fine-tuned model
instruct_model = AutoModelForSeq2SeqLM.from_pretrained(
    output_dir, torch_dtype="bfloat16", local_files_only=True
).to(device).eval()

In [ ]:

# Pick test sample
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

# Create summarization prompt
prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

# Tokenize input
# input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# Generate summary with original model (zero-shot)
original_model_outputs = original_model.generate(
    input_ids=input_ids,
    generation_config=GenerationConfig(max_new_tokens=200, num_beams=1)
)
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

# Generate summary with fine-tuned instruct model
instruct_model_outputs = instruct_model.generate(
    input_ids=input_ids,
    generation_config=GenerationConfig(max_new_tokens=200, num_beams=1)
)
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

# Print human, original, and instruct model summaries
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')


## Evaluate the Model Quantitatively (with ROUGE Metric)

In [ ]:
# Load ROUGE metric for evaluation
rouge = evaluate.load('rouge')

# Select first 10 test dialogues and human summaries
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

# Initialize lists to store model outputs
original_model_summaries = []
instruct_model_summaries = []

# Generate summaries for each dialogue
for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    # Original model summary
    original_model_outputs = original_model.generate(
        input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200)
    )
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    # Fine-tuned instruct model summary
    instruct_model_outputs = instruct_model.generate(
        input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200)
    )
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)

# Combine human and model summaries for comparison
zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))

# Convert to pandas DataFrame
df = pd.DataFrame(zipped_summaries, columns=['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df


In [ ]:
# Compute ROUGE scores for the original model
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,  # aggregate scores across all examples
    use_stemmer=True      # apply stemming for better matching
)

# Compute ROUGE scores for the fine-tuned instruct model
instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True
)

# Print ROUGE evaluation results
print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)


# Parameter Efficient Fine-Tuning (PEFT)

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

# Define the LoRA configuration
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,        #
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [ ]:
# Apply LoRA to the original model
peft_model = get_peft_model(original_model, lora_config)

# Print number of trainable parameters
print(print_number_of_trainable_model_parameters(peft_model))

## Train PEFT Adapter

In [ ]:
import time
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Training arguments (Seq2Seq version)
peft_training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=3,
    logging_steps=50,
    eval_strategy="epoch",   # <-- correct name in your version
    save_strategy="epoch",
)

# Initialize PEFT trainer
peft_trainer = Seq2SeqTrainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets.get("validation", None),
)

In [ ]:
# Start LoRA fine-tuning
peft_trainer.train()

# Define path to save the trained model
peft_model_path = "./peft-dialogue-summary-checkpoint-local"

# Save model and tokenizer
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

In [ ]:
from peft import PeftModel, PeftConfig

# Load pre-trained base model and tokenizer
peft_model_base = AutoModelForSeq2SeqLM.from_pretrained(
    "google/flan-t5-base",
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

# Load LoRA-adapted PEFT model from checkpoint, frozen for inference
peft_model = PeftModel.from_pretrained(
    peft_model_base,
    './peft-dialogue-summary-checkpoint-local',
    torch_dtype=torch.bfloat16,
    is_trainable=False
)


In [ ]:
print(print_number_of_trainable_model_parameters(peft_model))

### Evaluate the Model Qualitatively (Human Evaluation)

In [ ]:
# Move models to device
peft_model.to(device)

In [ ]:
# Load true original (pretrained) for baseline comparison
original_model = AutoModelForSeq2SeqLM.from_pretrained(
    "google/flan-t5-base", torch_dtype="bfloat16"
).to(device).eval()

# Load your fine-tuned model
instruct_model = AutoModelForSeq2SeqLM.from_pretrained(
    output_dir, torch_dtype="bfloat16", local_files_only=True
).to(device).eval()

In [ ]:
index = 200
dialogue = dataset['test'][index]['dialogue']
baseline_human_summary = dataset['test'][index]['summary']

# Prepare summarization prompt
prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

# Tokenize prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
# Generate summary using original model
original_model_outputs = original_model.generate(
    input_ids=input_ids,
    generation_config=GenerationConfig(max_new_tokens=200, num_beams=1)
)
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

# Generate summary using instruction-tuned model
instruct_model_outputs = instruct_model.generate(
    input_ids=input_ids,
    generation_config=GenerationConfig(max_new_tokens=200, num_beams=1)
)
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

# Generate summary using PEFT (LoRA) model
peft_model_outputs = peft_model.generate(
    input_ids=input_ids,
    generation_config=GenerationConfig(max_new_tokens=200, num_beams=1)
)
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

# Print comparison
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{baseline_human_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL:\n{peft_model_text_output}')


###  Evaluate the Model Quantitatively (with ROUGE Metric

In [ ]:

# Select first 10 test dialogues and their human summaries
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

# Lists to store model outputs
original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

# Generate summaries for each dialogue
for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)


    # Generate outputs for all models
    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    # Append outputs to lists
    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

# Combine human and model summaries into a DataFrame
zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries, peft_model_summaries))
df = pd.DataFrame(zipped_summaries, columns=['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])

df

In [ ]:
# Load ROUGE metric
rouge = evaluate.load('rouge')

# Compute ROUGE for original model
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

# Compute ROUGE for instruction-tuned model
instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

# Compute ROUGE for PEFT model
peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

# Print results
print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)
